<a href="https://colab.research.google.com/github/avitmsl/Number-plate-Detection-and-verifiation/blob/main/registration__number_dection_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** Camera Input **

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

**Obeject Detection**

In [ ]:
!pip install easyocr
!pip install imutils
!pip install pandas
!pip install opencv-python-headless==4.1.2.30
# MPS acceleration is available on MacOS 12.3+
!pip install torch torchvision torchaudio

import cv2
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr

In [ ]:

img = cv2.imread('/content/photo.jpg')
#img = cv2.imread('/content/photo.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#plt.imshow(cv2.cvtColor(gray,cv2.COLOR_BGR2RGB))

bfilter = cv2.bilateralFilter(gray,11,17,17)
edged=cv2.Canny(bfilter,30,200)
#plt.imshow(cv2.cvtColor(edged,cv2.COLOR_BGR2RGB))

keypoints = cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(keypoints)
contours = sorted(contours,key=cv2.contourArea,reverse=True)[:10]

location=None
for Contour in contours:
  approx = cv2.approxPolyDP(Contour,10,True)
  if len(approx) == 4:
    location = approx
    break

mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask,[location],0,255,-1)
new_image = cv2.bitwise_and(img,img,mask=mask)

(x,y) = np.where(mask==255)
(x1,y1) = (np.min(x),np.min(y))
(x2,y2) = (np.max(x),np.max(y))
cropped_image = gray[x1:x2+1, y1:y2+1]
plt.imshow(cv2.cvtColor(cropped_image,cv2.COLOR_BGR2RGB))

reader = easyocr.Reader(['en'])
result = reader.readtext(cropped_image)
#result
#cord=result[-1][1]
#cord
cord = result[-1][1]
#print("/n")
print(cord)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
res = cv2.putText(img, text= cord, org=(approx[0][0][0], approx[1][0][1]+60), fontFace = font, fontScale=1, color =(0,0,255), thickness=4, lineType=cv2.LINE_AA, bottomLeftOrigin=False)
res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0), 8)
plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))

searching plate number in dummy database

In [ ]:

my_dict = { 'WB 38 DE 0001': '''
            Owner_Name            : Avinash Kumar
            Driving_License       : WB3720210046667
            DL_Valid-till         : 18/12/2041
            Chessis_No            : MA3EADES00360056
            Engine_No             : K103N7162103
            Model_Name            : Honda City
            Registration_Date     : 30/05/2012
            Tax_Validation_upto   : 29/05/2027
            Vehicle_Class         : LMV
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 10/1/2024
            Registering_Authority : Asansol''',

            'WB 40 PG 9989': '''
            Owner_Name            : Debdas Dutta
            Driving_License       : WB3720210046668
            DL_Valid-till         : 20/05/2023
            Chessis_No            : FG3EAQSS003600099
            Engine_No             : K103N7162112
            Model_Name            : Nano
            Registration_Date     : 6/8/2013
            Tax_Validation_upto   : 5/8/2025
            Vehicle_Class         : MCWG
            Fuel_Type             : Diesel
            Insurance_valid_upto  : 20/02/2025
            Registering_Authority : MURSHIDABAD''',

            'WB 01 JE 8705': '''
            Owner_Name            : Subhashish Bhattacharya
            Driving_License       : WB3720210046669
            DL_Valid-till         : 25/09/2027
            Chessis_No            : AN3EAQSS00364096
            Engine_No             : S103Q7171220
            Model_Name            : Bolero
            Registration_Date     : 8/9/2012
            Tax_Validation_upto   : 7/9/2028
            Vehicle_Class         : LMV
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 25/08/2022
            Registering_Authority : Kolkata Beltala''',

            'WB 06 JA 0175': '''
            Owner_Name            : Pritam Ghosh
            Driving_License       : WB3720210046670
            DL_Valid-till         : 06/07/2021
            Chessis_No            : MA3EADES0036003
            Engine_No             : P205Q7161123
            Model_Name            : Tata Nexon
            Registration_Date     : 05/05/2014
            Tax_Validation_upto   : 04/05/2029
            Vehicle_Class         : MCWG
            Fuel_Type             : Diesel
            Insurance_valid_upto  : 27/07/2029
            Registering_Authority : Kolkata Ruby''',

            'WB 12 KQ 2967': '''
            Owner_Name            : Prabir Chatterjee
            Driving_License       : WB3720210046671
            DL_Valid-till         : 08/02/2035
            Chessis_No            : QA3EADA90360036
            Engine_No             : A1045B342122
            Model_Name            : Mahindra Thar
            Registration_Date     : 26/07/2010
            Tax_Validation_upto   : 25/07/2021
            Vehicle_Class         : LMV
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 28/05/2025
            Registering_Authority : Howrah''',

            'WB 13 BD 3059': '''
            Owner_Name            : Subrata Roy Chowdhury
            Driving_License       : WB3720210046672
            DL_Valid-till         : 09/05/2027
            Chessis_No            : WA32AF988945784
            Engine_No             : B1234D876549
            Model_Name            : Mahindra XUV
            Registration_Date     : 07/09/2011
            Tax_Validation_upto   : 06/09/2030
            Vehicle_Class         : MCWG
            Fuel_Type             : Diesel
            Insurance_valid_upto  : 05/05/2026
            Registering_Authority : Uluberia Arto''',

            'WB 38 A 4021': '''
            Owner_Name            : Deep Singh
            Driving_License       : WB3720210046673
            DL_Valid-till         : 02/12/2020
            Chessis_No            : FG98DG964386512
            Engine_No             : H2644C765435
            Model_Name            : Tyota Inova cresta
            Registration_Date     : 14/11/2012
            Tax_Validation_upto   : 13/11/2031
            Vehicle_Class         : LMV
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 06/08/2027
            Registering_Authority : Hugli''',

            'WB 38 A 1241':	'''
            Owner_Name:Jyotirmoy Bhattacharya
            Driving_License:WB3720210046674
            DL_Valid-till:01/01/2030
            Chessis_No:XV76AS674523109
            Engine_No:T6556F765432
            Model_Name:Tyouta Fortuner
            Registration_Date:11/12/2011
            Tax_Validation_upto:10/12/2035
            Vehicle_Class:MCWG
            Fuel_Type:Diesel
            Insurance_valid_upto:10/08/2023
            Registering_Authority:Srirampur Arto''',

            'WB 38 A 1242':	'''
            Owner_Name:Debabrata Ghosh
            Driving_License:WB3720210046675
            DL_Valid-till:12/12/2026
            Chessis_No:HB65WX549657088
            Engine_No:J7438H786539
            Model_Name:Maruti Brezza
            Registration_Date:17/02/2010
            Tax_Validation_upto:16/02/2021
            Vehicle_Class:LMV
            Fuel_Type:Petrol
            Insurance_valid_upto:04/06/2030
            Registering_Authority:Alipore''',

            'WB 38 A 1243':	'''
            Owner_Name:Anupam Banerjee
            Driving_License:WB3720210046676
            DL_Valid-till:27/10/2023
            Chessis_No:MH45AB579259026
            Engine_No:U5422K775543
            Model_Name:Hundai verna
            Registration_Date:10/01/2015
            Tax_Validation_upto:09/01/2028
            Vehicle_Class:MCWG
            Fuel_Type:Diesel
            Insurance_valid_upto:11/12/2027
            Registering_Authority:Bangaon Arto''',

            'WB 38 A 1244':	'''
            Owner_Name:Priyanka Mukherjee
            Driving_License:WB3720210046677
            DL_Valid-till:13/01/2026
            Chessis_No:KL76MJ0885690652
            Engine_No:L8765O784532
            Model_Name:Mahindra XUV 700
            Registration_Date:13/04/2014
            Tax_Validation_upto:12/04/2026
            Vehicle_Class:LMV
            Fuel_Type:Petrol
            Insurance_valid_upto:23/09/2032
            Registering_Authority:Barrackpore Arto''',

            'WB 38 A 1245':	'''
            Owner_Name:Pratima Banerjee
            Driving_License:WB3720210046678
            DL_Valid-till:07/11/2037
            Chessis_No:LO92KM8645896329
            Engine_No:M7432L907645
            Model_Name:Mahindra XUV 500
            Registration_Date:20/02/2013
            Tax_Validation_upto:19/02/2025
            Vehicle_Class:MCWG
            Fuel_Type:Petrol
            Insurance_valid_upto:15/08/2031
            Registering_Authority:Rampurhat Arto''',

            'WB 38 A 1246':	'''
            Owner_Name:Manjushree Bhattacharya
            Driving_License:WB3720210046679
            DL_Valid-till:05/01/2027
            Chessis_No:PK89DF0963527819
            Engine_No:B4569Y897032
            Model_Name:Swift Dezire
            Registration_Date:13/09/2012
            Tax_Validation_upto:12/09/2024
            Vehicle_Class:LMV
            Fuel_Type:Diesel
            Insurance_valid_upto:17/02/2026
            Registering_Authority:Bolpur Arto''',

            'WB 38 A 1247':	'''
            Owner_Name:Ipshita Sengupta
            Driving_License:WB3720210046680
            DL_Valid-till:26/02/2036
            Chessis_No:WK56FM9730289348
            Engine_No:X9865V743098
            Model_Name:Alto K10
            Registration_Date:18/03/2011
            Tax_Validation_upto:17/03/2028
            Vehicle_Class:MCWG
            Fuel_Type:Petrol
            Insurance_valid_upto:02/02/2035
            Registering_Authority:Jhargram Rto''',

            'WB 38 A 1248':	'''
            Owner_Name:Sreelekha Chatterjee
            Driving_License:WB3720210046681
            DL_Valid-till:15/03/2042
            Chessis_No:DH98NC7654329875
            Engine_No:G7865N876436
            Model_Name:Maruti Etriga
            Registration_Date:25/05/2010
            Tax_Validation_upto:24/05/2027
            Vehicle_Class:LMV	,Fuel_Type:Diesel
            Insurance_valid_upto:01/05/2033
            Registering_Authority:Nadia Rto''',

            'WB 38 A 1249':	'''
            Owner_Name:Abir Deb ,
            Driving_License:WB3720210046682
            DL_Valid-till:12/04/2021
            Chessis_No:ST8765UY378549630
            Engine_No:K8764D785470
            Model_Name:Tata Nano
            Registration_Date:26/07/2011
            Tax_Validation_upto:25/07/2028
            Vehicle_Class:MCWG
            Fuel_Type:Petrol
            Insurance_valid_upto:19/04/2036
            Registering_Authority:Birbhum Rto''',

            'WB 38 A 1250':	'''
            Owner_Name:Subir Sarkar
            Driving_License:WB3720210046683
            DL_Valid-till:17/06/2037
            Chessis_No:AK7463WK864903672
            Engine_No:W7649O09653
            Model_Name:Toyota Land Cruiser 300
            Registration_Date:03/09/2013
            Tax_Validation_upto:02/09/2030
            Vehicle_Class:LMV
            Fuel_Type:Diesel
            Insurance_valid_upto:25/05/2034
            Registering_Authority:Purulia Rto''',

            'WB 38 A 1251':	'''
            Owner_Name:Debashish Das
            Driving_License:WB3720210046684
            DL_Valid-till:19/05/2043
            Chessis_No:GH8743JK982364986
            Engine_No             : Y6739Y653784
            Model_Name            : Tata Tiago
            Registration_Date     : 14/07/2015
            Tax_Validation_upto   : 13/07/2025
            Vehicle_Class         : MCWG
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 26/07/2037
            Registering_Authority : Murshidabad Rto''',

            'WB 38 A 1252':	'''
            Owner_Name            : Joyeeta Das
            Driving_License       : WB3720210046685
            DL_Valid-till         : 20/01/2042
            Chessis_No            : RH8735OL984609648
            Engine_No             : O9723B765978
            Model_Name            : Mahindra
            Registration_Date     : XUV 700	20/04/2014
            Tax_Validation_upto   : 19/04/2026
            Vehicle_Class         : LMV
            Fuel_Type             : Diesel
            Insurance_valid_upto  : 21/09/2029
            Registering_Authority : Uttar Dinajpur Rto''',

            'WB 38 A 1253':	'''
            Owner_Name            : Anwesha Mukherjee
            Driving_License       : WB3720210046686
            DL_Valid-till         : 08/07/2039
            Chessis_No            : VJ9067GD876536480
            Engine_No             : H6745H764368
            Model_Name            : Mahindra XUV 500
            Registration_Date     : 07/10/2010
            Tax_Validation_upto   : 06/10/2033
            Vehicle_Class         : MCWG
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 02/10/2030
            Registering_Authority : Dakshin Dinajpur Rto''',

            'WB 38 A 1254':	'''
            Owner_Name            : Anushka Banerjee
            Driving_License       : WB3720210046687
            DL_Valid-till         : 09/08/2038
            Chessis_No            : KS8749XG763549065
            Engine_No             : D7649K974335
            Model_Name            : Swift Dezire
            Registration_Date     : 12/12/2011
            Tax_Validation_upto   : 11/12/2031
            Vehicle_Class         : LMV
            Fuel_Type             : Diesel
            Insurance_valid_upto  : 28/12/2033
            Registering_Authority : Cooch Behar''',

            'WB 38 A 1255':	'''
            Owner_Name:Subhojit Ray
            Driving_License       : WB3720210046688
            DL_Valid-till         : 21/09/2037
            Chessis_No            : AG6745FG586389468
            Engine_No             : U7620T763972
            Model_Name            : Alto K10
            Registration_Date     : 02/02/2013
            Tax_Validation_upto   : 01/02/2033
            Vehicle_Class         : MCWG
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 08/10/2038
            Registering_Authority : Malda Rto''',

            'WB 38 A 1256':	'''
            Owner_Name            : Debjit Roy
            Driving_License       : WB3720210046689
            DL_Valid-till         : 24/10/2036
            Chessis_No            : GY7845KF674289536
            Engine_No             : L6572W765889
            Model_Name            : Mahindra Thar
            Registration_Date     : 13/03/2015
            Tax_Validation_upto   : 12/03/2032
            Vehicle_Class         : LMV
            Fuel_Type             : Diesel
            Insurance_valid_upto  : 11/11/2041
            Registering_Authority : Bankura Rto''',

            'WB 38 A 1257':	'''
            Owner_Name            : Abhijit Ghosh
            Driving_License       : WB3720210046690
            DL_Valid-till         : 25/11/2035
            Chessis_No            : QT5375EH746389564
            Engine_No             : L9628V876889
            Model_Name            : Mahindra XUV
            Registration_Date     : 15/04/2014
            Tax_Validation_upto   : 14/04/2029
            Vehicle_Class         : MCWG
            Fuel_Type             : Petrol
            Insurance_valid_upto  : 13/03/2039
            Registering_Authority : Alipurduar''',

            'WB 38 A 1258':	'''
            Owner_Name            : Arup Das
            Driving_License       : WB3720210046691
            DL_Valid-till         : 26/12/2038
            Chessis_No            : FH6537DS763590547
            Engine_No             : X7638F762843
            Model_Name            : Tyota Inova cresta
            Registration_Date     : 25/05/2010
            Tax_Validation_upto   : 24/05/2028
            Vehicle_Class         : LMV
            Fuel_Type             : Diesel
            Insurance_valid_upto  : 15/01/2040
            Registering_Authority : Jalpaiguri Rto''',

}





val=my_dict.get(cord)
print(val)

